#### Pre-Processing

In [1]:
# Parse a PDF as well as a email invoice
from src.utils import preprocess_aor, preprocess_invoice
# pdf_path = "database/aor/AOR-sample1.pdf"
# preprocess_aor(aor_dir="database/aor")

# preprocess_invoice(invoice_dir="database/invoice")

Preprocessing AORs...


#### Search & Query

In [2]:
# Chat with the preprocessed AOR for real-time chat based on RAG 
from src.aor import load_aors
from src.search import search_aor_with_item, search_aor_with_no


aor_list = load_aors()

# Quick routing: Item-based & Number-baseds
no_query = "DH928"
matching_aors = search_aor_with_no(no_query, aor_list)


print("Item-based AOR searching:")
for aor in matching_aors[:1]:
    print(f"AOR No: {aor.no}")
    print(f"Description: {aor.description}")
    print(f"Items: {aor.items}")
    print(f"Budgets: {aor.budgets}")
    print(f"Expiry Date: {aor.expiry_date}")
    print("---")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Item-based AOR searching:
AOR No: DH928/6/4
Description: Approval of requirement specifications for the implementation of Crew Mobile Digital Services Lite (CMDS-Lite) for First Flotilla, specifically for a Proof of Concept (PoC) trial on two Frigates (FFSes).
Items: ['Hardware for 2 FFSes', 'Software and Licenses for 2 FFSes', 'IT man-effort and Professional Services']
Budgets: [36000.0, 10000.0, 78000.0]
Expiry Date: 2024-05-16
---


#### RAG-based chat 

In [2]:
from src.agent import RagAgent 

# Example usage
agent = RagAgent()
user_message = "Tell me about AOR DH928"
# user_message = "Look for AOR which covers toilet paper"
# user_message = "What is the invoice no.01141883 about?"
agent_response = agent.chat(user_message)

user_message = "What is the invoice no.01141833 about?"
agent_response = agent.chat(user_message)

# user_message = "Really?"
# agent_response = agent.chat(user_message)

Calling Function:  search_aor_with_no  | Query:  DH928
Retrived Information: Found AORDH928/6 /4 related to the query number DH928
Calling Function:  direct_answer  | Query:  The AOR with number DH928/6 /4 has a budget for Hardware of 36000.0 for 2 FFSes, a budget for Software and Licenses of 10000.0 for 2 FFSes, and a budget for IT man-effort and Professional Services of 78000.0. This AOR is set to expire on 2025-06-16 and it involves the Approval of Requirement Specifications for the Implementation of Crew Mobile Digital Services Lite (CMDS-Lite) for the First Flotilla.
Direct Answer
Final Answer: AOR DH928 pertains to the Approval of Requirement Specifications for the Implementation of Crew Mobile Digital Services Lite (CMDS-Lite) for the First Flotilla. The budget allocated includes 36,000.0 for hardware for 2 FFSes, 10,000.0 for software and licenses, and 78,000.0 for IT man-effort and professional services. The AOR expires on June 16, 2025.
Calling Function:  search_invoice_with_

#### Invoice Processor

In [5]:
# this is the last query to solve, before the front-end deployment 
# query = "Is the invoice no.01141833 covered by any of the AORs?"
# query = "What is the invoice no.01141883 about?"

# use_message = "Is the invoice no.01141833 covered by any of the AORs?"
agent_response = agent.chat("Is the invoice no.01141833 covered by any of the AORs?")

Retrived Information: Found Invoice01141833 related to the query number 01141833
Direct Answer
Final Answer: The invoice no. 01141833 is not mentioned or covered by AOR DH928.


In [6]:
# Chainlit front-end interface with logo change now
